In [ ]:
import os

from langchain_community.document_loaders import PubMedLoader
from langchain_community.retrievers import PubMedRetriever

from langchain_community.document_loaders import TextLoader
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

from langchain_groq.chat_models import ChatGroq
from langchain_ollama import ChatOllama

import uuid

from dotenv import load_dotenv, find_dotenv

# Load the API keys from .env
load_dotenv(find_dotenv(), override=True)


from src.vectordb.create_vectordb import PinconeVectorDb

In [17]:
pc = PinconeVectorDb()
pc.create_pinecone_index(index_name='test-pubmed')

Index Stats:
{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [20]:
embeddings = HuggingFaceEmbeddings(model_name=os.getenv('EMBEDDING_MODEL_NAME'))
vectorstore = PineconeVectorStore(index=pc.index, embedding=embeddings)
vectorstore

In [19]:
loader = PubMedLoader('cardiology', load_max_docs=20)
documents = loader.load()
documents

[Document(metadata={'uid': '39569514', 'Title': 'Response of Blood Pressure to Renal Denervation Is Not Associated With Genetic Variants.', 'Published': '2024-11-21', 'Copyright Information': ''}, page_content='BACKGROUND: Renal denervation lowers blood pressure (BP) in patients with uncontrolled hypertension. We conducted an unbiased genomic screen to identify genetic variants that may associate with BP response to renal denervation (RDN).\nMETHODS: Patients (n=268) with uncontrolled resistant hypertension (baseline BP, 166±21/90±15 mm\u2005Hg) who underwent endovascular RDN using the Symplicity catheter (Medtronic, Inc, Santa Rosa, CA) were included. Reduction in 24-hour ambulatory systolic BP was assessed at 6 months and divided into 2 groups: above and below the median response of 6.0 mm\u2005Hg, taking preintervention 24-hour ambulatory BP and regression to the mean into account. Whole exome sequencing assessing 249\u2005669 variants, was conducted using Illumina NovaSeq technolog

In [23]:
len(documents)

20

In [21]:
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)
docs = doc_splitter.split_documents(documents)
docs

[Document(metadata={'uid': '39569514', 'Title': 'Response of Blood Pressure to Renal Denervation Is Not Associated With Genetic Variants.', 'Published': '2024-11-21', 'Copyright Information': ''}, page_content='BACKGROUND: Renal denervation lowers blood pressure (BP) in patients with uncontrolled hypertension. We conducted an unbiased genomic screen to identify genetic variants that may associate with BP response to renal denervation (RDN).'),
 Document(metadata={'uid': '39569514', 'Title': 'Response of Blood Pressure to Renal Denervation Is Not Associated With Genetic Variants.', 'Published': '2024-11-21', 'Copyright Information': ''}, page_content='METHODS: Patients (n=268) with uncontrolled resistant hypertension (baseline BP, 166±21/90±15 mm\u2005Hg) who underwent endovascular RDN using the Symplicity catheter (Medtronic, Inc, Santa Rosa, CA) were included. Reduction in 24-hour ambulatory systolic BP was assessed at 6 months and divided into 2 groups: above and below the median res

In [22]:
len(docs)

95

In [27]:
uuids = [str(uuid.uuid4()) for _ in range(len(documents))]

vectorstore.add_documents(documents=documents, ids=uuids)

vectorstore.add_documents(docs)

['de2199c2-325a-4440-9268-2ba175637696',
 'd8cde668-b784-480d-a10e-08ce1a106a7d',
 '3e2d8279-8c66-40de-8f7f-35e96a4295fd',
 '2ae523e3-6de6-4143-927d-b10ca2692ef3',
 '6af68e0d-7391-430f-abb4-eb1f996e9ca7',
 '1ded7a43-ce5d-46f6-b093-1da84ca2f19a',
 'ac1b0168-0619-4822-9485-4a34de29c9f0',
 '2e51886e-5576-4368-a1b9-a101a547e46b',
 'de3a7005-b8c7-4c93-a0a1-e03e086f884d',
 '8ccee045-7e55-46ac-b209-04ddf20c4842',
 '407180a3-5e38-468e-aaf1-e77ea1abc43f',
 'eb598500-0703-453f-aff2-43e7ce7de46a',
 '45a74870-5423-408c-b412-0b3ba48730c4',
 '632f7d40-34e8-40e7-8472-f70253ed5d3b',
 'a05e4ab6-9337-49e2-84e9-15f5ab10f074',
 'ad42a858-33b1-4920-94da-0681ce8913e1',
 '869ded5c-a298-4de4-9c7b-9e7896132c8c',
 '54c5c034-817b-4f71-ab65-e2200e5c1db8',
 'c0cfc372-df20-4b05-9907-b539568857e2',
 '00c3314c-fe2f-4ffb-ac44-35a8d3c3e63f',
 '3d1fffd6-3e5b-4b53-b956-5289bb912e60',
 '0de567d7-b80e-451d-874c-0a0ce142b395',
 '0b452193-7659-4004-ab33-5c42b31fa946',
 'c0e79786-0d1f-453a-9d58-b8103ebbec0b',
 'cf1ed096-53a6-

In [42]:
vectorstore

In [28]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000021C113FEC80>, search_type='mmr', search_kwargs={'k': 3})

In [30]:
retriever.invoke("latest trends in cardiology")

[Document(metadata={'Copyright Information': '', 'Published': '2024-11-21', 'Title': 'Long-Term Management of Right Ventricular Outflow Tract Dysfunction in Repaired Tetralogy of Fallot: A Scientific Statement From the American Heart Association.', 'uid': '39569497'}, page_content='late outcomes has become an urgent priority. This American Heart Association scientific statement provides an update on the current state of knowledge of the pathophysiology, methods of surveillance, risk stratification, and latest available therapies, including transcatheter and surgical pulmonary valve replacement strategies, as well as management of life-threatening arrhythmias. It reviews emerging evidence on the roles of comorbidities and patient-reported outcomes and their impact on quality of life.'),
 Document(metadata={'Copyright Information': '', 'Published': '2024-11-21', 'Title': 'S100A1ct: A Synthetic Peptide Derived From S100A1 Protein Improves Cardiac Performance and Survival in Preclinical He

In [ ]:
# llm = ChatGroq(model="llama-3.1-70b-versatile",
#                       stop_sequences="[end]",
#                       temperature=0.)



llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0.,
    
)

In [34]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [44]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What are the latest trends in cardiology?"})

results

{'input': 'What are the latest trends in cardiology?',
 'context': [Document(metadata={'Copyright Information': '', 'Published': '2024-11-21', 'Title': 'Long-Term Management of Right Ventricular Outflow Tract Dysfunction in Repaired Tetralogy of Fallot: A Scientific Statement From the American Heart Association.', 'uid': '39569497'}, page_content='late outcomes has become an urgent priority. This American Heart Association scientific statement provides an update on the current state of knowledge of the pathophysiology, methods of surveillance, risk stratification, and latest available therapies, including transcatheter and surgical pulmonary valve replacement strategies, as well as management of life-threatening arrhythmias. It reviews emerging evidence on the roles of comorbidities and patient-reported outcomes and their impact on quality of life.'),
  Document(metadata={'Copyright Information': 'Copyright © 2024, Kumar et al.', 'Published': '2024-10-21', 'Title': 'Etiology, Predispos

In [38]:
question_answer_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021C363EA6B0>, async_client=<groq.resources.chat.completions.AsyncCompletio

In [40]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000021C113FEC80>, search_type='mmr', search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved